# Classificação dos personagens dos Simpsons

Alunos: Alexandre Borges Baccarini Júnior - 2515520
        Leonardo
        Mateus

## Objetivo

Desenvolver um sistema inteligente para classificação de personagens da série Os Simpsons utilizando os classificadores k-NN e SVM.


### Baixar as bibliotecas

In [4]:
#RODAR UMA VEZ PARA BAIXAR AS BIBLIOTECAS

%pip install numpy pandas Pillow scikit-learn keras imblearn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 KB 1.0 MB/s eta 0:00:00a 0:00:01
  Using cached scikit_learn-1.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.5 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.7.0
    Uninstalling scikit-learn-1.7.0:
      Successfully uninstalled scikit-learn-1.7.0
Note: you may need to restart the kernel to use updated packages.


### Extração de Características

In [5]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from keras.applications.vgg16 import VGG16, preprocess_input

def load_img(img_path, target_size):
    img = Image.open(img_path)
    img = img.convert('RGB')
    img = img.resize(target_size)
    return img

def extract_features(directory):
    model = VGG16(weights='imagenet', include_top=False)
    features = []
    labels = []
    for filename in os.listdir(directory):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            img_path = os.path.join(directory, filename)
            label = directory.split(os.sep)[-1]
            try:
                image = load_img(img_path, target_size=(224, 224))
                image = np.array(image)
                image = np.expand_dims(image, axis=0)
                image = preprocess_input(image)
                feature = model.predict(image).flatten()
                features.append(feature)
                labels.append(label)
            except Exception as e:
                print(f"Error processing image: {img_path}")
                print(f"Exception: {e}")
    return features, labels

# TREINO
bart_dir = "./imgs/Train/bart"
homer_dir = "./imgs/Train/homer"
lisa_dir = "./imgs/Train/lisa"
maggie_dir = "./imgs/Train/maggie"
marge_dir = "./imgs/Train/marge"

bart_features, bart_labels = extract_features(bart_dir)
homer_features, homer_labels = extract_features(homer_dir)
lisa_features, lisa_labels = extract_features(lisa_dir)
maggie_features, maggie_labels = extract_features(maggie_dir)
marge_features, marge_labels = extract_features(marge_dir)

all_features = lisa_features + maggie_features + marge_features
all_labels =lisa_labels + maggie_labels + marge_labels

data = {'class': all_labels}
for i in range(len(all_features[0])):
    data[f'feature_{i+1}'] = [feat[i] for feat in all_features]

df = pd.DataFrame(data)

csv_train_path = "./data/Train/feats.csv"
df.to_csv(csv_train_path, index=False)

# TESTE
bart_dir = "./imgs/Valid/bart"
homer_dir = "./imgs/Valid/homer"
lisa_dir = "./imgs/Valid/lisa"
maggie_dir = "./imgs/Valid/maggie"
marge_dir = "./imgs/Valid/marge"

bart_features, bart_labels = extract_features(bart_dir)
homer_features, homer_labels = extract_features(homer_dir)
lisa_features, lisa_labels = extract_features(lisa_dir)
maggie_features, maggie_labels = extract_features(maggie_dir)
marge_features, marge_labels = extract_features(marge_dir)

all_features = lisa_features + maggie_features + marge_features
all_labels =lisa_labels + maggie_labels + marge_labels

data = {'class': all_labels}
for i in range(len(all_features[0])):
    data[f'feature_{i+1}'] = [feat[i] for feat in all_features]

df = pd.DataFrame(data)

csv_test_path = "./data/Valid/feats.csv"
df.to_csv(csv_test_path, index=False)


2025-06-28 14:58:17.322627: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-28 14:58:17.442823: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-28 14:58:17.550712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751133497.646691   50918 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751133497.679113   50918 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751133497.909076   50918 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 517ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

### Treinamento e Avaliação dos Modelos
#### k-NN

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV
import pandas as pd

try:
    from imblearn.over_sampling import SMOTE
    smote_available = True
except ImportError:
    smote_available = False
    print("SMOTE não disponível - usando RandomOverSampler")
    from imblearn.over_sampling import RandomOverSampler

# Carrega os dados
csv_train_path = "./data/Train/feats.csv"
df_train = pd.read_csv(csv_train_path)

# Análise de distribuição
print("Distribuição das classes no treino:")
print(df_train['class'].value_counts())

# Separa os dados
X_train = df_train.drop('class', axis=1)
y_train = df_train['class']

# Codifica as classes
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Normaliza os dados
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Balanceamento
if smote_available:
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train_encoded)
else:
    ros = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train_encoded)

# Grid de parâmetros
valores_grid = [
    {'n_neighbors': [3, 5, 7, 9], 'metric': ['euclidean'], 'weights': ['distance']},
    {'n_neighbors': [3, 5, 7, 9], 'metric': ['manhattan'], 'weights': ['distance']}
]

# Cria e treina o modelo
modelo = KNeighborsClassifier()
gridKNN = GridSearchCV(estimator=modelo, param_grid=valores_grid, cv=5, verbose=1, n_jobs=-1, scoring='f1_macro')
gridKNN.fit(X_resampled, y_resampled)

# Resultados
print(f"\nMelhor acurácia: {gridKNN.best_score_}")
print(f"Melhor K: {gridKNN.best_estimator_.n_neighbors}")
print(f"Melhor distância: {gridKNN.best_estimator_.metric}")
print(f"Melhores pesos: {gridKNN.best_estimator_.weights}")

Distribuição das classes no treino:
class
lisa      33
maggie    30
marge     24
Name: count, dtype: int64
Fitting 5 folds for each of 8 candidates, totalling 40 fits

Melhor acurácia: 0.6797999603881957
Melhor K: 3
Melhor distância: manhattan
Melhores pesos: distance


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd

# Carrega os dados de teste
csv_test_path = "./data/Valid/feats.csv"
df_test = pd.read_csv(csv_test_path)

# Análise de distribuição
print("\nDistribuição das classes no teste:")
print(df_test['class'].value_counts())

# Separa os dados
X_test = df_test.drop('class', axis=1)
y_test = df_test['class']

# Codifica as classes
y_test_encoded = le.transform(y_test)

# Normaliza com o mesmo scaler do treino
X_test_scaled = scaler.transform(X_test)

# Cria o modelo com melhores parâmetros
modelo = KNeighborsClassifier(
    n_neighbors=gridKNN.best_estimator_.n_neighbors,
    metric=gridKNN.best_estimator_.metric,
    weights=gridKNN.best_estimator_.weights
)

# Treina com dados resampleados
modelo.fit(X_resampled, y_resampled)

# Predição
y_pred_encoded = modelo.predict(X_test_scaled)

# Avaliação
print("\nAvaliação do Modelo:")
print(f"Acurácia: {accuracy_score(y_test_encoded, y_pred_encoded)}")
print("\nRelatório de Classificação:")
target_names = le.classes_
print(classification_report(y_test_encoded, y_pred_encoded, target_names=target_names))

print("\nMatriz de Confusão:")
print(confusion_matrix(y_test_encoded, y_pred_encoded))


Distribuição das classes no teste:
class
lisa      13
maggie    12
marge     11
Name: count, dtype: int64

Avaliação do Modelo:
Acurácia: 0.5277777777777778

Relatório de Classificação:
              precision    recall  f1-score   support

        lisa       0.58      0.54      0.56        13
      maggie       0.48      0.92      0.63        12
       marge       1.00      0.09      0.17        11

    accuracy                           0.53        36
   macro avg       0.69      0.52      0.45        36
weighted avg       0.68      0.53      0.46        36


Matriz de Confusão:
[[ 7  6  0]
 [ 1 11  0]
 [ 4  6  1]]


#### VSM

In [ ]:
#VSM - code

### Comparação dos Modelos

In [ ]:
#comparação -code

### Análise de Erros

In [ ]:
#Análise de Erros - code